In [3]:
using GAP

Adding path /root/.julia/packages/GAP/W20PN/gap/.libs to DL_LOAD_PATH
 ┌───────┐   GAP 4.11dev-1183-gf1f5dc5 of today
 │  GAP  │   https://www.gap-system.org
 └───────┘   Architecture: x86_64-pc-linux-gnu-julia64-kv7
 Configuration:  gmp 6.1.2, Julia GC, Julia 1.2.0
 Loading the library and packages ...
 Packages:   GAPDoc 1.6.3, PrimGrp 3.3.2, SmallGrp 1.4.1, TransGrp 2.0.4
 Try '??help' for help. See also '?copyright', '?cite' and '?authors'


In [6]:
# コーナーキューブを定義
Uc = @gap ( 2, 5, 7, 4) * (10,34,26,18);
Lc = @gap (10,13,15,12) * ( 4,20,44,37);
Fc = @gap (18,21,23,20) * ( 7,28,42,13);
Rc = @gap (26,29,31,28) * ( 5,36,45,21);
Bc = @gap (34,37,39,36) * ( 2,12,47,29);
Dc = @gap (42,45,47,44) * (15,23,31,39);
coner_action = [Uc, Uc^2, Uc^3, Lc, Lc^2, Lc^3, Fc, Fc^2, Fc^3, Rc, Rc^2, Rc^3, Bc, Bc^2, Bc^3, Dc, Dc^2, Dc^3]
coner_cube = (@gap Group)(Uc,Lc,Fc,Rc,Bc,Dc)

# エッジキューブの定義
Ue = @gap ( 1, 3, 8, 6) * ( 9,33,25,17) * (11,35,27,19);
Le = @gap ( 9,11,16,14) * ( 1,17,41,40) * ( 6,22,46,35);
Fe = @gap (17,19,24,22) * ( 6,25,43,16) * ( 8,30,41,11);
Re = @gap (25,27,32,30) * ( 3,38,43,19) * ( 8,33,48,24);
Be = @gap (33,35,40,38) * ( 3, 9,46,32) * ( 1,14,48,27);
De = @gap (41,43,48,46) * (14,22,30,38) * (16,24,32,40);
edge_action = [Ue, Ue^2, Ue^3, Le, Le^2, Le^3, Fe, Fe^2, Fe^3, Re, Re^2, Re^3, Be, Be^2, Be^3, De, De^2, De^3]
edge_cube = (@gap Group)(Ue,Le,Fe,Re,Be,De)

GAP: <permutation group with 6 generators>

In [91]:
function create_table(actions::Array{Main.ForeignGAP.MPtr,1}; search_max = 20)
    e = @gap ()
    depth_table = [0 => [e], 1 => []] # こいつを保存する
    all_set = [e]
    depth = 1
    current = 1
    next = 0
    all = 1
    while true
        s = depth_table[depth][2][current]
        current -= 1
        for a ∈ actions
            ns = s * a
            if ns ∉ all_set
                push!(depth_table[depth + 1][2], ns)
                push!(all_set, ns)
                next += 1
                all += 1
            end
        end
        if current == 0
            println("{ depth: $(depth), created_state: $(next), all_state: $(all) }")
            current = next
            next = 0
            depth += 1
            push!(depth_table, depth => [])
        end
        
        search_max < depth && return depth_table
    end
    depth_table
end

create_table (generic function with 1 method)

In [92]:
coner_table = create_table(coner_action; search_max=3)

{ depth: 1, created_state: 18, all_state: 19 }
{ depth: 2, created_state: 243, all_state: 262 }
{ depth: 3, created_state: 3240, all_state: 3502 }


5-element Array{Pair{Int64,Array{Any,1}},1}:
 0 => [GAP: ()]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [94]:
edge_table = create_table(edge_action; search_max=3)

{ depth: 1, created_state: 18, all_state: 19 }
{ depth: 2, created_state: 243, all_state: 262 }
{ depth: 3, created_state: 2874, all_state: 3136 }


5-element Array{Pair{Int64,Array{Any,1}},1}:
 0 => [GAP: ()]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [102]:
fifteen_puzzle_code(initial_state) = begin
  sum_of_path_cost = 0
  solution = []
  limit = Inf16
  path_cost(parent, child, action) = 1
  is_goal(state) = begin
    for i ∈ 1:15 
      i ≠ state[i] && return false 
    end 
    true
  end
    
  actions(state) = begin
    list = []    
    for i ∈ 1:4
      for j ∈ 1:4
        x, y = i-1, j-1
        if state[x * 4 + y + 1] ≠ 16
          continue
        end
        
        if i ≠ 1
          push!(list, ((x - 1) * 4 + y + 1, x * 4 + y + 1))
        end

        if i ≠ 4
          push!(list, ((x + 1) * 4 + y + 1, x * 4 + y + 1))
        end

        if j ≠ 1
          push!(list, (x * 4 + y, x * 4 + y + 1))
        end

        if j ≠ 4
          push!(list, (x * 4 + y + 2, x * 4 + y + 1))
        end
        
      end
    end
        
    list
  end
    
  child(parent, action) = begin
    c = Array(parent)
    tmp = c[action[1]]
    c[action[1]] = c[action[2]]
    c[action[2]] = tmp
    c
  end

  heuristic(c) = begin
    h = 0
    for i ∈ 1:16
      c[i] == 16 && continue
      x1, y1 = (c[i] - 1) ÷ 4, (c[i] - 1) % 4
      x2, y2 = (i - 1) ÷ 4, (i - 1) % 4
      h += abs(x1 - x2) + abs(y1 - y2)
    end
    h
  end

  function search(state, path, search_limit, search_path)
    (search_limit < sum_of_path_cost + heuristic(state)) && return "cutoff" => Array{Tuple{Int64,Int64},1}()
    
    is_goal(state) && return "found" => search_path

    cutoff = false
    list = actions(state)
    
    for action ∈ list
      c = child(state, action)
      
      c ∈ path && continue
      
      push!(search_path, action)
      push!(path, c)
      cost = path_cost(state, c, action)

      sum_of_path_cost += cost
      result = search(c, path, search_limit, search_path)
            
      result[1] == "found" && return result
      
      pop!(path)
      pop!(search_path)
      sum_of_path_cost -= cost
      cutoff = true
    end
    
    cutoff && return "cutoff" => Array{Tuple{Int64,Int64},1}()
        
    "not found" => Array{Tuple{Int64,Int64},1}()
  end

  limit = heuristic(initial_state)
  search_step = 1
  search_path = [initial_state]
  # 100回で打ち止め
  for i in 1:100
    result = search(initial_state, search_path, limit, Array{Tuple{Int64,Int64},1}())
    if result[1] == "found"
      println("solution is $(result[2])")
      return result[2]
    end

    if result[1] == "not found"
      println("no solution")
      return result[2]
    end

    println("when search step is $(search_step) and limit is $(limit), solution is not found.")
    search_step += 1
    limit += 1
  end
end

fifteen_puzzle_code (generic function with 1 method)

In [106]:
initial_state = [2,7,4,15,13,9,3,8,1,6,16,14,12,5,10,11]
solution = fifteen_puzzle_code(initial_state)

process(initial_state, solution) = begin
    next = initial_state
    for s in solution
        tmp = next[s[1]]
        next[s[1]] = next[s[2]]
        next[s[2]] = tmp
      print("""
              action: $(s)
              $(next[1]) $(next[2]) $(next[3]) $(next[4])
              $(next[5]) $(next[6]) $(next[7]) $(next[8])
              $(next[9]) $(next[10]) $(next[11]) $(next[12])
              $(next[13]) $(next[14]) $(next[15]) $(next[16])
              """)  
    end
end
process(initial_state, solution)

when search step is 1 and limit is 30, solution is not found.
when search step is 2 and limit is 31, solution is not found.
when search step is 3 and limit is 32, solution is not found.
when search step is 4 and limit is 33, solution is not found.
when search step is 5 and limit is 34, solution is not found.
when search step is 6 and limit is 35, solution is not found.
when search step is 7 and limit is 36, solution is not found.
when search step is 8 and limit is 37, solution is not found.
solution is Tuple{Int64,Int64}[(10, 11), (9, 10), (5, 9), (6, 5), (10, 6), (14, 10), (13, 14), (9, 13), (5, 9), (6, 5), (7, 6), (8, 7), (4, 8), (3, 4), (2, 3), (6, 2), (10, 6), (11, 10), (12, 11), (8, 12), (7, 8), (3, 7), (2, 3), (1, 2), (5, 1), (6, 5), (10, 6), (14, 10), (15, 14), (11, 15), (10, 11), (14, 10), (15, 14), (16, 15), (12, 16), (11, 12), (15, 11), (16, 15)]
action: (10, 11)
2 7 4 15
13 9 3 8
1 16 6 14
12 5 10 11
action: (9, 10)
2 7 4 15
13 9 3 8
16 1 6 14
12 5 10 11
action: (5, 9)
2 7 4

In [101]:
solution

38-element Array{Tuple{Int64,Int64},1}:
 (10, 11)
 (9, 10) 
 (5, 9)  
 (6, 5)  
 (10, 6) 
 (14, 10)
 (13, 14)
 (9, 13) 
 (5, 9)  
 (6, 5)  
 (7, 6)  
 (8, 7)  
 (4, 8)  
 ⋮       
 (10, 6) 
 (14, 10)
 (15, 14)
 (11, 15)
 (10, 11)
 (14, 10)
 (15, 14)
 (16, 15)
 (12, 16)
 (11, 12)
 (15, 11)
 (16, 15)